In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.style as mplstyle
mplstyle.use('fast')
import ompy as om
om.print_status()

Load the matrix. OMpy defaults to numpy `.npz` files, but MAMA `.m`, ROOT files and others are supported.

In [21]:
mat = om.Matrix.from_path('120Sn_raw_without_bg.npz')

In [22]:
mat.plot()

We don't want to include counts after the diagonal. The diagonal is defined as $Ex = Eg + n\sigma(Eg, Ex)$. As a simplification, we use a line-by-eye to create a mask.

In [30]:
line = om.Line(p1=(450.0, 250.0), slope=1)
mask = line.below(mat)
mat[mask] = 0
mat.plot()

Get the response matrix. 

In [31]:
resp = om.response.Response.from_db('Oscar2020')

The discrete response matrix `R` and the smoothing matrix for $\sigma_{\text{Eg}}$ `G` are constructed by specializing the response to a matrix

In [36]:
R = resp.specialize_like(mat)
G = resp.gaussian_like(mat)

In [37]:
R.plot()
G.plot()

Since $R$ maps $\mu \to \delta$ and $G$ maps $\delta \to \nu$ (or $\mu \to \eta$), the map $\mu \to \nu$ is given by $GR$.

In [38]:
GR = G@R
GR.plot()

As creating $G$ and $R$ is so common, the unfolders using the `Unfolder` interace accepts a `Response` instance.
Unfolding using Guttormsen iteration is fast and converges to maximum likelihood if noise is low and the response is perfect. In order to use the mapping trick, unfolding to $\eta$ to lift the degeneracy of $G$, we specify the unfolder to use the response $R$ to unfold. The folding by $G$ is handled behind the scenes.

In [52]:
unf = om.unfolding.Guttormsen.from_response(resp, mat)
res = unf.unfold(mat, R='R', iterations=1000)

In [53]:
res.plot_comparison(vmin=0, vmax=1e4)

There are many negative counts, indicating that the response is wrong (here too low compton).